# Model Troll Tweets

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import HiveContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StopWordsRemover

import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
spark = SparkSession.builder.appName('rus_trolls').getOrCreate()

# Change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '8g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])

# Print spark configuration settings
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.yarn.appMasterEnv.MKL_NUM_THREADS', '1'),
 ('spark.driver.memory', '4g'),
 ('spark.sql.queryExecutionListeners',
  'com.cloudera.spark.lineage.NavigatorQueryListener'),
 ('spark.ui.proxyBase', '/proxy/application_1547750003855_6397'),
 ('spark.ui.killEnabled', 'true'),
 ('spark.lineage.log.dir', '/var/log/spark/lineage'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'md01.rcc.local,md02.rcc.local'),
 ('spark.eventLog.dir', 'hdfs://nameservice1/user/spark/applicationHistory'),
 ('spark.yarn.am.extraLibraryPath',
  '/opt/cloudera/parcels/CDH-6.1.0-1.cdh6.1.0.p0.770702/lib/hadoop/lib/native'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.executor.extraLibraryPath',
  '/opt/cloudera/parcels/CDH-6.1.0-1.cdh6.1.0.p0.770702/lib/hadoop/lib/native'),
 ('spark.driver.extraLibraryPath',
  '/opt/cloudera/parcels/CDH-6.1.0-1.cdh6.1.0.p0.770702/lib/hadoop/lib/native'),
 ('spark

In [7]:
sc = spark.sparkContext
hc = HiveContext(sc)

# Read data

In [8]:
hive_context = HiveContext(sc)
df = hive_context.table("tywang.trolls_non_trolls")
df.show(10)

+--------+--------------------+
|   class|          tweet_text|
+--------+--------------------+
|innocent|@angelayee hey gi...|
|innocent|i HATE the rain i...|
|innocent|@jasontryfon Thin...|
|innocent|o gosh need help ...|
|innocent|Aww man, it sound...|
|innocent|watching the dark...|
|innocent|just learned I ca...|
|innocent|Uuuuuuuuuuuugh an...|
|innocent|@jacobawhite Have...|
|innocent|@tkorte20 peeps I...|
+--------+--------------------+
only showing top 10 rows



In [10]:
df.groupBy("class").count().show()

+--------+-------+
|   class|  count|
+--------+-------+
|   troll|1358472|
|innocent|1600000|
+--------+-------+



# Transform data

In [12]:
tokenizer = Tokenizer(inputCol="tweet_text", outputCol="words")
df_words = tokenizer.transform(df)
df_words = df_words.drop("tweet_text")
df_words.show(5)

+--------+--------------------+
|   class|               words|
+--------+--------------------+
|innocent|[@angelayee, hey,...|
|innocent|[i, hate, the, ra...|
|innocent|[@jasontryfon, th...|
|innocent|[o, gosh, need, h...|
|innocent|[aww, man,, it, s...|
+--------+--------------------+
only showing top 5 rows



In [15]:
# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_content")
df_words_removed_sw = remover.transform(df_words)

# Drop the redundant source column
df_words_removed_sw = df_words_removed_sw.drop("words")
df_words_removed_sw.show(5)

+--------+--------------------+
|   class|    filtered_content|
+--------+--------------------+
|innocent|[@angelayee, hey,...|
|innocent|[hate, rain, hate...|
|innocent|[@jasontryfon, th...|
|innocent|[o, gosh, need, h...|
|innocent|[aww, man,, sound...|
+--------+--------------------+
only showing top 5 rows



In [17]:
# Learn a mapping from words to Vectors
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=5, minCount=0, inputCol="filtered_content", outputCol="wordVectors")
w2VM = word2Vec.fit(df_words_removed_sw)
nlpdf = w2VM.transform(df_words_removed_sw)

Py4JJavaError: An error occurred while calling o607.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: 
Aborting TaskSet 16.0 because task 0 (partition 0)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 0.0 in stage 16.0 (TID 212, hd03.rcc.local, executor 4): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:619)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.co

In [ ]:
# Split data into train and test
splits = nlpdf.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

train_df.show(1)

In [ ]:
# Set parameters for Logistic Regression
lgr = LogisticRegression(maxIter=10, featuresCol="wordVectors", labelCol="class")

# Fit the model to the data.
lgrm = lgr.fit(train_df)

# Given a dataset, predict each point's label, and show the results.
predictions = lgrm.transform(test_df)

In [ ]:
# print evaluation metrics
evaluator = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))